<a href="https://colab.research.google.com/github/seirah-yang/Colabwork/blob/main/(V2)_%EC%BD%94%EC%8A%A4%EB%8B%A5_%EC%BD%94%EC%8A%A4%ED%94%BC_%EB%82%98%EC%8A%A4%EB%8B%A5_S%26P500_%EB%B0%B0%EB%8B%B9%EC%A0%95%EB%B3%B4_from_yahoofinance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##실행하면 만들어지는 파일

./data/dividends_all_companies_5y.csv : 일자별 배당 (티커, 이름, 날짜, 주당배당금)

./data/dividends_yearly_sum.csv : 연도별 합계 배당

./data/tickers_not_found.csv : 야후에 존재하지 않는 티커

./data/tickers_with_no_dividends.csv : 최근 5년 배당 기록이 없는 유효 티커

In [ ]:
#  올인원: 유니버스 생성 -> 정제/존재성 필터 -> 배당 수집 -> 저장 (국내+해외)
#  출력:
#   ./data/dividends_all_companies_5y.csv        (일자별 배당)
#   ./data/dividends_yearly_sum.csv             (연도별 합계)
#   ./data/tickers_bad_format.csv               (형식 불량 삭제 목록)
#   ./data/tickers_not_found.csv                (야후 미존재/정지 추정 목록)
# ======================================================================
!pip -q install yfinance pandas lxml tqdm

In [1]:
import os, re, time, logging, traceback
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

In [7]:
# -------------------- 설정 --------------------
OUT_DIR = "./data"
os.makedirs(OUT_DIR, exist_ok=True)

DIV_CSV       = os.path.join(OUT_DIR, "dividends_all_companies_5y.csv")
YEARLY_CSV    = os.path.join(OUT_DIR, "dividends_yearly_sum.csv")
NOT_FOUND_CSV = os.path.join(OUT_DIR, "tickers_not_found.csv")
BAD_FORMAT_CSV= os.path.join(OUT_DIR, "tickers_bad_format.csv")

LAST_N_YEARS       = 5
START_DATE         = (datetime.now() - timedelta(days=365*LAST_N_YEARS)).strftime("%Y-%m-%d")
MAX_WORKERS_CHECK  = 4     # 야후 존재성 확인 동시실행 개수
MAX_WORKERS_DIV    = 4     # 배당 수집 동시실행 개수
REQUEST_SLEEP      = 0.1  # 요청 간 짧은 지연(레이트리밋 방지)
RETRY              = 3

INCLUDE_KRX        = True
INCLUDE_SP500      = True
INCLUDE_NASDAQ100  = True
EXTRA_TICKERS      = []    # 예: ['TSLA','NVDA','7203.T','005930.KS']


In [8]:
# yfinance 로그 잠재우기(원하면 주석 처리)
logging.getLogger("yfinance").setLevel(logging.CRITICAL)

[WARN] dividends fetch fail: 031980.KS (피에스케이홀딩스) -> Invalid comparison between dtype=datetime64[ns, Asia/Seoul] and Timestamp
[WARN] dividends fetch fail: 031820.KS (아이티센씨티에스) -> Invalid comparison between dtype=datetime64[ns, Asia/Seoul] and Timestamp


In [9]:
def build_universe_sp500() -> pd.DataFrame:
    """위키 S&P500 구성종목(표 변화 방어)"""
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    tables = pd.read_html(url, header=0)
    for tbl in tables:
        if isinstance(tbl.columns, pd.MultiIndex):
            tbl.columns = [str(x[0]) for x in tbl.columns]
        else:
            tbl.columns = [str(x) for x in tbl.columns]
        sym_candidates = [c for c in tbl.columns if any(k in c.lower() for k in ["symbol","ticker"])]
        if not sym_candidates:
            continue
        sym_col = sym_candidates[0]
        name_candidates = [c for c in tbl.columns if any(k in c.lower() for k in ["security","company","name"])]
        name_col = name_candidates[0] if name_candidates else sym_col
        out = (tbl[[sym_col, name_col]].rename(columns={sym_col:"yahoo_ticker", name_col:"name"}))
        out["yahoo_ticker"] = out["yahoo_ticker"].astype(str).str.strip()
        out["name"] = out["name"].astype(str).str.strip()
        out = out.dropna().drop_duplicates().reset_index(drop=True)
        if "yahoo_ticker" in out.columns and len(out) > 100:
            return out
    raise RuntimeError("S&P500 파싱 실패(표 구조 변경 가능).")

def build_universe_nasdaq100() -> pd.DataFrame:
    """위키 NASDAQ-100 구성종목(표 변화 방어)"""
    url = "https://en.wikipedia.org/wiki/Nasdaq-100"
    tables = pd.read_html(url, header=0)
    for tbl in tables:
        if isinstance(tbl.columns, pd.MultiIndex):
            tbl.columns = [str(x[0]) for x in tbl.columns]
        else:
            tbl.columns = [str(x) for x in tbl.columns]
        sym_candidates = [c for c in tbl.columns if any(k in c.lower() for k in ["ticker","symbol"])]
        if not sym_candidates:
            continue
        sym_col = sym_candidates[0]
        name_candidates = [c for c in tbl.columns if any(k in c.lower() for k in ["company","name","security"])]
        name_col = name_candidates[0] if name_candidates else sym_col
        out = (tbl[[sym_col, name_col]].rename(columns={sym_col:"yahoo_ticker", name_col:"name"}))
        out["yahoo_ticker"] = out["yahoo_ticker"].astype(str).str.strip()
        out["name"] = out["name"].astype(str).str.strip()
        out = out.dropna().drop_duplicates().reset_index(drop=True)
        if "yahoo_ticker" in out.columns and len(out) > 50:
            return out
    raise RuntimeError("NASDAQ-100 파싱 실패(표 구조 변경 가능).")

def build_universe_extra(tickers) -> pd.DataFrame:
    rows = [{"yahoo_ticker": t.strip(), "name": t.strip()} for t in tickers if str(t).strip()]
    return pd.DataFrame(rows) if rows else pd.DataFrame(columns=["yahoo_ticker","name"])

# -------------------- 유니버스 생성 --------------------
univ_parts = []
try:
    #if INCLUDE_KRX:       univ_parts.append(build_universe_krx())
    if INCLUDE_SP500:     univ_parts.append(build_universe_sp500())
    if INCLUDE_NASDAQ100: univ_parts.append(build_universe_nasdaq100())
    if EXTRA_TICKERS:     univ_parts.append(build_universe_extra(EXTRA_TICKERS))
except Exception as e:
    print("[WARN] 유니버스 일부 생성 실패:", e)

if not univ_parts:
    raise RuntimeError("유니버스가 비었습니다. INCLUDE_* 설정을 확인하세요.")

univ = pd.concat(univ_parts, ignore_index=True).drop_duplicates().reset_index(drop=True)
print("유니버스 총 종목 수:", len(univ))

# -------------------- 티커 정제 + 야후 존재성 필터 --------------------
# KRX(.KS/.KQ) → 숫자 6자리만 유지, 해외는 공백 제거
u = univ.copy()
u["yahoo_ticker"] = u["yahoo_ticker"].astype(str).str.strip()
krx_mask = u["yahoo_ticker"].str.endswith((".KS",".KQ"), na=False)

def normalize_krx_ticker(t):
    base, suf = t.rsplit('.', 1)
    base_digits = re.sub(r"\D", "", base)  # 숫자만
    if len(base_digits) != 6:
        return None
    return f"{base_digits}.{suf}"

u.loc[krx_mask, "normalized"] = u.loc[krx_mask, "yahoo_ticker"].apply(normalize_krx_ticker)
u.loc[~krx_mask, "normalized"] = u.loc[~krx_mask, "yahoo_ticker"].str.replace(r"\s+", "", regex=True)

bad_fmt = u[u["normalized"].isna()][["yahoo_ticker","name"]]
if len(bad_fmt):
    bad_fmt.to_csv(BAD_FORMAT_CSV, index=False, encoding="utf-8-sig")
    print(f"형식 불량 제거: {len(bad_fmt)}개 → {BAD_FORMAT_CSV}")

u = u[u["normalized"].notna()].copy()
u["yahoo_ticker"] = u["normalized"]
u = u.drop(columns=["normalized"]).drop_duplicates().reset_index(drop=True)

print("정제 후 후보 티커 수:", len(u))

_exists_cache = {}
def yahoo_exists(ticker: str) -> bool:
    if ticker in _exists_cache:
        return _exists_cache[ticker]
    ok = False
    for _ in range(RETRY):
        try:
            hist = yf.Ticker(ticker).history(period="1mo", auto_adjust=False)
            time.sleep(REQUEST_SLEEP)
            if not hist.empty:
                ok = True
                break
            div = yf.Ticker(ticker).dividends
            time.sleep(REQUEST_SLEEP)
            if div is not None and not div.empty:
                ok = True
                break
        except Exception:
            time.sleep(0.4)
    _exists_cache[ticker] = ok
    return ok

exist_flags = []
with ThreadPoolExecutor(max_workers=MAX_WORKERS_CHECK) as ex:
    futures = {ex.submit(yahoo_exists, t): (t, n) for t, n in u[["yahoo_ticker","name"]].itertuples(index=False)}
    for f in tqdm(as_completed(futures), total=len(futures), desc="야후 존재 확인"):
        t, n = futures[f]
        ok = False
        try:
            ok = f.result()
        except Exception:
            ok = False
        exist_flags.append((t, ok))

exist_df  = pd.DataFrame(exist_flags, columns=["yahoo_ticker","exists"])
not_found = u.merge(exist_df, on="yahoo_ticker", how="left")
not_found = not_found[not_found["exists"] == False][["yahoo_ticker","name"]]
if len(not_found):
    not_found.to_csv(NOT_FOUND_CSV, index=False, encoding="utf-8-sig")
    print(f"야후 미존재/정지 추정: {len(not_found)}개 → {NOT_FOUND_CSV}")

univ_ok = u.merge(exist_df, on="yahoo_ticker", how="left")
univ_ok = univ_ok[univ_ok["exists"] == True][["yahoo_ticker","name"]].drop_duplicates().reset_index(drop=True)
print("최종 수집 대상 티커 수(univ_ok):", len(univ_ok))


[WARN] dividends fetch fail: 015360.KS (INVENI) -> Invalid comparison between dtype=datetime64[ns, Asia/Seoul] and Timestamp
[WARN] dividends fetch fail: 000660.KS (SK하이닉스) -> Invalid comparison between dtype=datetime64[ns, Asia/Seoul] and Timestamp
[WARN] dividends fetch fail: 023150.KS (MH에탄올) -> Invalid comparison between dtype=datetime64[ns, Asia/Seoul] and Timestamp
[WARN] dividends fetch fail: 023450.KS (동남합성) -> Invalid comparison between dtype=datetime64[ns, Asia/Seoul] and Timestamp
[WARN] dividends fetch fail: 029530.KS (신도리코) -> Invalid comparison between dtype=datetime64[ns, Asia/Seoul] and Timestamp
유니버스 총 종목 수: 522
정제 후 후보 티커 수: 522
[WARN] dividends fetch fail: 028050.KS (삼성E&A) -> Invalid comparison between dtype=datetime64[ns, Asia/Seoul] and Timestamp


야후 존재 확인:   0%|          | 1/522 [00:00<03:24,  2.54it/s]

[WARN] dividends fetch fail: 005320.KS (온타이드) -> Invalid comparison between dtype=datetime64[ns, Asia/Seoul] and Timestamp


야후 존재 확인:   5%|▍         | 26/522 [00:02<00:59,  8.37it/s]

[WARN] dividends fetch fail: 011560.KS (세보엠이씨) -> Invalid comparison between dtype=datetime64[ns, Asia/Seoul] and Timestamp
[WARN] dividends fetch fail: 030720.KS (동원수산) -> Invalid comparison between dtype=datetime64[ns, Asia/Seoul] and Timestamp
[WARN] dividends fetch fail: 024740.KS (한일단조) -> Invalid comparison between dtype=datetime64[ns, Asia/Seoul] and Timestamp
[WARN] dividends fetch fail: 024070.KS (WISCOM) -> Invalid comparison between dtype=datetime64[ns, Asia/Seoul] and Timestamp


...

[WARN] dividends fetch fail: 025900.KS (동화기업) -> Invalid comparison between dtype=datetime64[ns, Asia/Seoul] and Timestamp


야후 존재 확인:  95%|█████████▌| 497/522 [00:27<00:01, 16.55it/s]

[WARN] dividends fetch fail: 011090.KS (에넥스) -> Invalid comparison between dtype=datetime64[ns, Asia/Seoul] and Timestamp
[WARN] dividends fetch fail: 025950.KS (동신건설) -> Invalid comparison between dtype=datetime64[ns, Asia/Seoul] and Timestamp
[WARN] dividends fetch fail: 007460.KS (에이프로젠) -> Invalid comparison between dtype=datetime64[ns, Asia/Seoul] and Timestamp


야후 존재 확인: 100%|██████████| 522/522 [00:28<00:00, 18.31it/s]

야후 미존재/정지 추정: 31개 → ./data/tickers_not_found.csv[WARN] dividends fetch fail: 017960.KS (한국카본) -> Invalid comparison between dtype=datetime64[ns, Asia/Seoul] and Timestamp

최종 수집 대상 티커 수(univ_ok): 491


In [10]:
# -------------------- 배당 수집 --------------------
START_TS = pd.Timestamp(START_DATE)

def fetch_dividends(ticker: str, name: str):
    for attempt in range(RETRY):
        try:
            t = yf.Ticker(ticker)
            div = t.dividends
            time.sleep(REQUEST_SLEEP)
            if div is None or div.empty:
                return []
            div = div[div.index >= START_TS]
            return [
                {
                    "ticker": ticker,
                    "name": name,
                    "div_date": idx.strftime("%Y-%m-%d"),
                    "dividend_per_share": float(val)
                }
                for idx, val in div.items()
            ]
        except Exception as e:
            if attempt < RETRY - 1:
                time.sleep(0.5 * (attempt + 1))
            else:
                # 마지막 실패만 경고
                print(f"[WARN] dividends fetch fail: {ticker} ({name}) -> {e}")
                return []

all_rows = []
with ThreadPoolExecutor(max_workers=MAX_WORKERS_DIV) as ex:
    futures = {ex.submit(fetch_dividends, r["yahoo_ticker"], r["name"]): (r["yahoo_ticker"], r["name"])
               for _, r in univ_ok.iterrows()}
    for f in tqdm(as_completed(futures), total=len(futures), desc="배당 수집"):
        try:
            rows = f.result()
            if rows:
                all_rows.extend(rows)
        except Exception as e:
            t, n = futures[f]
            print(f"[WARN] future failed: {t} ({n}) -> {e}")

df_div = pd.DataFrame(all_rows, columns=["ticker","name","div_date","dividend_per_share"])
if not df_div.empty:
    df_div = df_div.sort_values(["ticker","div_date"]).drop_duplicates().reset_index(drop=True)

# 배당 전무한 유효 티커 기록
no_div_tickers = set(univ_ok["yahoo_ticker"]) - set(df_div["ticker"])
print("최근 5년 배당 없음 티커 수:", len(no_div_tickers))
if no_div_tickers:
    univ_ok[univ_ok["yahoo_ticker"].isin(no_div_tickers)] \
        .to_csv(os.path.join(OUT_DIR, "tickers_with_no_dividends.csv"), index=False, encoding="utf-8-sig")
    print("→ 목록 저장:", os.path.join(OUT_DIR, "tickers_with_no_dividends.csv"))


[WARN] dividends fetch fail: 002680.KS (한탑) -> Invalid comparison between dtype=datetime64[ns, Asia/Seoul] and Timestamp


배당 수집:   0%|          | 0/491 [00:00<?, ?it/s]

[WARN] dividends fetch fail: 025880.KS (케이씨피드) -> Invalid comparison between dtype=datetime64[ns, Asia/Seoul] and Timestamp
[WARN] dividends fetch fail: 025550.KS (한국선재) -> Invalid comparison between dtype=datetime64[ns, Asia/Seoul] and Timestamp
[WARN] dividends fetch fail: 024950.KS (삼천리자전거) -> Invalid comparison between dtype=datetime64[ns, Asia/Seoul] and Timestamp
[WARN] dividends fetch fail: 014100.KS (메디앙스) -> Invalid comparison between dtype=datetime64[ns, Asia/Seoul] and Timestamp
[WARN] dividends fetch fail: 024940.KS (PN풍년) -> Invalid comparison between dtype=datetime64[ns, Asia/Seoul] and Timestamp


배당 수집:   0%|          | 1/491 [00:04<36:40,  4.49s/it]

[WARN] dividends fetch fail: ABBV (AbbVie) -> Invalid comparison between dtype=datetime64[ns, America/New_York] and Timestamp
[WARN] dividends fetch fail: 004780.KS (대륙제관) -> Invalid comparison between dtype=datetime64[ns, Asia/Seoul] and Timestamp
[WARN] dividends fetch fail: 024910.KS (경창산업) -> Invalid comparison between dtype=datetime64[ns, Asia/Seoul] and Timestamp
[WARN] dividends fetch fail: 024830.KS (세원물산) -> Invalid comparison between dtype=datetime64[ns, Asia/Seoul] and Timestamp


...

[WARN] dividends fetch fail: MELI (Mercado Libre) -> Invalid comparison between dtype=datetime64[ns, America/New_York] and Timestamp


배당 수집: 100%|█████████▉| 490/491 [09:02<00:00,  1.61it/s]

[WARN] dividends fetch fail: ODFL (Old Dominion Freight Line) -> Invalid comparison between dtype=datetime64[ns, America/New_York] and Timestamp


배당 수집: 100%|██████████| 491/491 [09:02<00:00,  1.11s/it]

[WARN] dividends fetch fail: TRI (Thomson Reuters) -> Invalid comparison between dtype=datetime64[ns, America/New_York] and Timestamp
최근 5년 배당 없음 티커 수: 485
→ 목록 저장: ./data/tickers_with_no_dividends.csv


In [11]:
# -------------------- 저장 --------------------
df_div["collected_at"] = datetime.now().strftime("%Y-%m-%d %H:%M")
df_div.to_csv(DIV_CSV, index=False, encoding="utf-8-sig")
print("일자별 배당 저장:", DIV_CSV, "행수:", len(df_div))

if not df_div.empty:
    df_div["year"] = df_div["div_date"].str[:4]
    yearly = (df_div.groupby(["ticker","name","year"], as_index=False)["dividend_per_share"]
              .sum()
              .rename(columns={"dividend_per_share":"dividend_sum_per_share"}))
    yearly.to_csv(YEARLY_CSV, index=False, encoding="utf-8-sig")
    print("연도별 합계 저장:", YEARLY_CSV, "행수:", len(yearly))
else:
    print("연도별 합계 생성 스킵(배당 데이터 없음)")

일자별 배당 저장: ./data/dividends_all_companies_5y.csv 행수: 0
연도별 합계 생성 스킵(배당 데이터 없음)


##사용 팁

###속도 조절:
MAX_WORKERS_CHECK / MAX_WORKERS_DIV와 REQUEST_SLEEP 값을 환경에 맞게 조절. 너무 높이면 야후에서 레이트리밋 걸릴 수 있다.

###검증:
해외 배당 확실한 티커(AAPL, MSFT, KO, PG 등)를 EXTRA_TICKERS에 넣고 먼저 동작 확인

###위키 파싱 실패 시:
셀이 한 번에 실패하면 그 셀만 다시 실행
위키 테이블 구조가 바뀌거나 일시적 네트워크 이슈가 있을 수 있다.

In [ ]:
# -------------------- 유니버스 생성 함수 --------------------
def build_universe_krx() -> pd.DataFrame:
    """KIND에서 KOSPI/KOSDAQ 전 종목 로딩 -> yahoo_ticker/name 생성"""
    # KOSPI
    url_kospi  = "http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13"
    df_kospi   = pd.read_html(url_kospi, header=0, encoding="euc-kr")[0]
    df_kospi["시장구분"] = "KOSPI"
    # KOSDAQ
    url_kosdaq = "http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&marketType=kosdaq&searchType=13"
    df_kosdaq  = pd.read_html(url_kosdaq, header=0, encoding="euc-kr")[0]
    df_kosdaq["시장구분"] = "KOSDAQ"

    df_all = pd.concat([df_kospi, df_kosdaq], ignore_index=True)
    df_all = df_all.rename(columns={"회사명": "종목명", "종목코드": "종목코드"})
    df_all["종목코드"] = df_all["종목코드"].astype(str).str.zfill(6)
    # 숫자 6자리만 유지(이상치 제거)
    df_all = df_all[df_all["종목코드"].str.match(r"^\d{6}$")]

    def to_yahoo(row):
        code = row["종목코드"]
        mkt  = str(row["시장구분"]).upper()
        return f"{code}.KS" if mkt == "KOSPI" else (f"{code}.KQ" if mkt == "KOSDAQ" else f"{code}.KS")

    df_all["yahoo_ticker"] = df_all.apply(to_yahoo, axis=1)
    df_all["name"] = df_all["종목명"]
    return df_all[["yahoo_ticker","name"]].drop_duplicates().reset_index(drop=True)